<a href="https://colab.research.google.com/github/Sandesh-L/Bluelight-NLP/blob/main/NLP_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests

# note that CLIENT_ID refers to 'personal use script' and SECRET_TOKEN to 'token'
auth = requests.auth.HTTPBasicAuth('LNClsG-sU5-tywzvjSsspg', 'JPlScE5YA1PLNsFmg1xLFRl8CWSt3Q')

# here we pass our login method (password), username, and password
data = {'grant_type': 'password',
        'username': 'nlp-project1',
        'password': 'project-nlp123'}

# setup our header info, which gives reddit a brief description of our app
headers = {'User-Agent': 'MyBot/0.0.1'}

# send our request for an OAuth token
res = requests.post('https://www.reddit.com/api/v1/access_token',
                    auth=auth, data=data, headers=headers)

# convert response to JSON and pull access_token value
TOKEN = res.json()['access_token']

# add authorization to our headers dictionary
headers = {**headers, **{'Authorization': f"bearer {TOKEN}"}}

# while the token is valid (~2 hours) we just add headers=headers to our requests
requests.get('https://oauth.reddit.com/api/v1/me', headers=headers)

In [ ]:
#get posts for /r/OpiatesRecovery/top
res = requests.get("https://oauth.reddit.com/r/OpiatesRecovery/top",
                   headers=headers)

print(res.json())  # let's see what we get

In [ ]:
for post in res.json()['data']['children']:
  print(post['data']['title'])

In [ ]:
import pandas as pd

df = pd.DataFrame()  # initialize dataframe
params = {'limit':1000}

#get posts for /r/OpiatesRecovery/top
res = requests.get("https://oauth.reddit.com/r/OpiatesRecovery/top",
                   headers=headers,
                   params=params,
                   )

# loop through each post retrieved from GET request
for post in res.json()['data']['children']:
    # append relevant data to dataframe
    df = df.append({
        'subreddit': post['data']['subreddit'],
        'title': post['data']['title'],
        'selftext': post['data']['selftext'],
        'upvote_ratio': post['data']['upvote_ratio'],
        'ups': post['data']['ups'],
        'downs': post['data']['downs'],
        'score': post['data']['score']
    }, ignore_index=True)

In [ ]:
print(df.shape)
df.head(10)


In [39]:
!pip install asyncpraw


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.5/196.5 KB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 35.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 KB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 KB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 KB 16.1 MB/s eta 0:00:00


In [43]:
import asyncpraw

reddit = asyncpraw.Reddit(
    client_id="LNClsG-sU5-tywzvjSsspg",
    client_secret="JPlScE5YA1PLNsFmg1xLFRl8CWSt3Q",
    user_agent="nlp-project1",
)

In [65]:

print(reddit.read_only)


True


In [69]:
from datetime import datetime
date = datetime.now()
date

datetime.datetime(2023, 3, 22, 18, 37, 58, 61865)

In [71]:
subreddit = await reddit.subreddit("OpiatesRecovery")
# print(len(subreddit.hot()))
date = datetime.now
posts = subreddit.hot(limit=10000)
count = 0
async for submission in posts:
    count += 1
    print(submission.title)

Relapsed yesterday evening..
Vivitrol straight up stopped working. Snorting dope and getting high as though I didn't even take a shot of natrexone. How?!?!
Not that many people know but I have been clean for 4 whole days
Hey, i wanted to share my story, this has been a very regulated and obsessive-compulsive journey for me.
Anyone have any tips on how to forget a phone number?
Hi.I posted here before to help people find best supplements and quit opiates.but now I need ALL your help
Help me understand my sister, please
i met a counsellor
Day 19
Thursday March 2nd Daily Check In
Update
relapsed on those counterfeit m30's.
Sister has obscene amount of prescription painkillers and won’t acknowledge that she’s addicted. How can I help her?
Anyone here been through fent withdrawals?
Slept walk the other night?
Triggered
2 Weeks In -- Getting Clean and AA
What Happens Next
Wednesday March 1st Daily Check In
How to get help for a loved when everything is against you
Felt so CONFIDENT and beaut

In [67]:
count

1001